<a href="https://colab.research.google.com/github/wolfzxcv/ml-examples/blob/master/TSP_using_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Components of a Genetic Algorithm**

* Selection: Choosing individuals from the population to create offspring.
* Crossover: Combining parts of two individuals to create a new individual.
* Mutation: Introducing small random changes to individuals to maintain genetic diversity.

In [263]:
from random import randint, choice
from math import exp

# A very high value to represent no direct path between cities
INT_MAX = 2147483647

# Number of cities in TSP
V = 5

# Names of the 5 cities
GENES = "ABCDE"

# Initial population size (potential solutions) for the algorithm
POP_SIZE = 10

# Max number of generations
max_gen = 20

# A list to store the population of individuals
population = []

In [264]:
# individual solution (a possible path)
class Individual:
    def __init__(self, gnome, fitness):
        # A string representing the path (e.g., "012340")
        self.gnome = gnome
        # The total distance of the path
        self.fitness = fitness


# Generate a random number between start and end-1
def rand_num(start, end):
    return randint(start, end - 1)  # Adjust for Python's zero-based indexing


# Generates a valid initial path (gnome) starting and ending at the first city
def create_gnome():
    gnome = str(choice(range(V)))
    while len(gnome) < V:
        temp = rand_num(0, V)
        if str(temp) not in gnome:
            gnome += str(temp)
    return gnome


# Perform crossover between two parents to create offspring
def crossover(parent1, parent2):
    child_p1 = ""
    child_p2 = ""

    geneA = int(rand_num(1, V))
    geneB = int(rand_num(1, V))

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        child_p1 += parent1.gnome[i]

    child_p2 = [item for item in parent2.gnome if item not in child_p1]

    child = child_p1 + ''.join(child_p2)

    return child


# Mutates a gnome by swapping two random cities in the path
def mutated_gene(gnome):
    gnome = list(gnome)
    while True:
        r = rand_num(1, V)
        r1 = rand_num(1, V)
        if r1 != r:
            temp = gnome[r]
            gnome[r] = gnome[r1]
            gnome[r1] = temp
            break
    return ''.join(gnome)


# Calculates the fitness of a gnome by summing the distances between continuous cities in the path
def calculate_fitness(gnome):
    # distance_matrix represents the distances between cities
    # The fitness value is the total distance traveled
    # Summing up the distances between continuous cities along the path
    distance_matrix = [
        [0, 2, INT_MAX, 12, 5],
        [2, 0, 4, 8, INT_MAX],
        [INT_MAX, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, INT_MAX, 3, 10, 0],
    ]
    fitness = 0
    for i in range(V - 1):
        fitness += distance_matrix[int(gnome[i])][int(gnome[i + 1])]
    return fitness

In [265]:
# Main function for TSP problem.
def tsp_util():
    global population, max_gen  # Declare population and max_gen as global

    # Current generation number
    gen = 1

    print("Initial population: \nGNOME FITNESS VALUE")
    # Populating the GNOME pool.
    for i in range(POP_SIZE):
        gnome = create_gnome()
        fitness = calculate_fitness(gnome)
        print(gnome, fitness)
        population.append(Individual(gnome, fitness))

    # Iteration to perform
    # population crossing and gene mutation.
    while gen <= max_gen:
      population.sort(key=lambda x: x.fitness)
      new_population = []

      # Elitism: Carry over the best individuals to the next generation
      elites_count = int(POP_SIZE * 0.5)  # Keep top 50% of the population as elites
      elites = population[:elites_count]
      new_population.extend(elites)

      # Selection, Crossover, and Mutation
      for _ in range(POP_SIZE - elites_count):
          parent1 = choice(population)
          parent2 = choice(population)
          child_gnome = crossover(parent1, parent2)
          mutated_gnome = mutated_gene(child_gnome)
          new_fitness = calculate_fitness(mutated_gnome)
          new_population.append(Individual(mutated_gnome, new_fitness))

      population = new_population

      print(f"\nGeneration {gen}")
      print("GNOME FITNESS VALUE")

      for ind in population:
          print(f"{ind.gnome} {ind.fitness}")

      gen += 1

    # Print the smallest fitness value (shortest path) after the last iteration
    # Find the individual with the minimum fitness
    best_individual = min(population, key=lambda x: x.fitness)
    best_gnome = best_individual.gnome
    best_fitness = best_individual.fitness
    print(f"\nShortest path (smallest fitness value in the last iteration): {best_fitness}")

    # Decode the gnome (visiting order)
    city_map = {GENES[i]: i for i in range(len(GENES))}

    # Convert index string to city names
    result = "".join([GENES[int(city)] for city in best_gnome][::-1])  # List comprehension with reverse

    # Print the solution with city names (including starting city)
    print(f"Visiting order (city names): {result}{result[0]}")  # Add starting city at the end

if __name__ == "__main__":
    tsp_util()

Initial population: 
GNOME FITNESS VALUE
42013 2147483660
40321 24
20134 2147483667
04231 19
10342 27
10423 13
13042 28
04213 20
20341 4294967316
10423 13

Generation 1
GNOME FITNESS VALUE
10423 13
10423 13
04231 19
04213 20
40321 24
30412 2147483668
42031 2147483670
34021 2147483666
10432 20
03124 27

Generation 2
GNOME FITNESS VALUE
10423 13
10423 13
04231 19
04213 20
10432 20
43201 2147483662
31204 2147483664
01423 2147483655
34012 21
12403 24

Generation 3
GNOME FITNESS VALUE
10423 13
10423 13
04231 19
04213 20
10432 20
24310 23
42130 27
01234 19
13042 28
23140 2147483663

Generation 4
GNOME FITNESS VALUE
10423 13
10423 13
04231 19
01234 19
04213 20
21340 27
04312 27
32401 13
01423 2147483655
10324 20

Generation 5
GNOME FITNESS VALUE
10423 13
10423 13
32401 13
04231 19
01234 19
20314 4294967314
01243 19
01234 19
04312 27
41023 4294967299

Generation 6
GNOME FITNESS VALUE
10423 13
10423 13
32401 13
04231 19
01234 19
01423 2147483655
03142 2147483670
42103 21
02143 4294967308
04312 